In [1]:
import numpy as np
import os

from sklearn.utils import shuffle
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
face_data=[]
labels=[]
dataset_path='./data/'

class_id=0   #label for the given Title
names={}     #Mapping between id<->name

#Data Preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        # Create a mapping between class_id and name
        names[class_id]=fx[:-4]
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        
        #Create labels for the class
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)
        
face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))
#trainset=np.concatenate((face_dataset,face_labels),axis=1)

In [3]:
X,Y=shuffle(face_dataset,face_labels,random_state=1)

In [4]:
X=X/255.0
X=X.reshape((-1,224,224,3))

In [5]:
Y=np_utils.to_categorical(Y)

In [6]:
print(X.shape)
print(Y.shape)

(20, 224, 224, 3)
(20, 2)


In [7]:
from keras.layers import *
from keras.models import Sequential

In [8]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(Y.shape[1],activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 128)      

In [9]:
from keras import optimizers

In [10]:
adam=optimizers.adam(lr=1e-4)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [11]:
model.fit(X,Y,shuffle=True,epochs=10,batch_size=2,validation_split=0.2)

Train on 16 samples, validate on 4 samples
Epoch 1/10
16/16 [==============================] - 17s 1s/step - loss: 0.7202 - accuracy: 0.3750 - val_loss: 0.6713 - val_accuracy: 0.7500
Epoch 2/10
16/16 [==============================] - 16s 1s/step - loss: 0.6692 - accuracy: 0.5625 - val_loss: 0.6378 - val_accuracy: 1.0000
Epoch 3/10
16/16 [==============================] - 16s 1s/step - loss: 0.6279 - accuracy: 0.9375 - val_loss: 0.5882 - val_accuracy: 1.0000
Epoch 4/10
16/16 [==============================] - 17s 1s/step - loss: 0.5706 - accuracy: 1.0000 - val_loss: 0.5188 - val_accuracy: 1.0000
Epoch 5/10
16/16 [==============================] - 16s 1s/step - loss: 0.4914 - accuracy: 1.0000 - val_loss: 0.4115 - val_accuracy: 1.0000
Epoch 6/10
16/16 [==============================] - 16s 1s/step - loss: 0.3861 - accuracy: 1.0000 - val_loss: 0.2700 - val_accuracy: 1.0000
Epoch 7/10
16/16 [==============================] - 18s 1s/step - loss: 0.2538 - accuracy: 0.9375 - val_loss: 0.1427 

In [13]:
with open("model.json", "w") as file:
    file.write(model.to_json())
model.save_weights("model.h5")

In [14]:
from pathlib import Path

In [15]:
count=0
idx_to_name={}
p=Path('data/')
dirs=p.glob('*')
for d in dirs:
    idx_to_name[count]=str(d).split('\\')[-1][:-4]
    count+=1

In [16]:
idx_to_name

{0: 'imran', 1: 'masoom'}